# Data Collection, Merging and Cleaning

## Change in Mobility during COVID19 Pandemy

### 1. Introduction

This study is conducted to investigate the change in mobility behaviour during pandemy. In the scope of this study, Mobility Report published by Google is utilized. To be able to understand the reasons behind the change in mobility, various data from multiple sources are collected.

There are wide variety of data available on the internet, however, finding and collecting useful data requires extensive work. Some important challenges as follows: 

<ul>
<li>Most available data are not useful.</li>
<li>The available data should match with the existing data in terms of time interval, locations and so on.</li>
<li>Many data providers charge for their service.</li>
</ul>

In this work, Google Mobility Data, GDP per capita, weather data, COVID19 case records, city development index, traffic congestion index, fuel prices are collected, all dataframes are cleaned and merged in a single dataframe. 

### 2. Importing Libraries

Firstly, the libraries that we will need to manipulate data should be imported, such as numpy and pandas. Data visualization libraries like Matplotlib and seaborn may be usefull on the road. Furthermore, some data can be collected in json format from an URL. Most municipalities publishes their data in this way. Thus, we should also import additional libraries like json, urllib, etc.

In [132]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import urllib.request as urllib
import re

<br><br><br>

### 3. Collecting and Merging Data

In this section, data is collected from multiple sources and merged with the main dataframe after data cleaning.

#### 3.1. Global Mobility Data

<b>Source:</b> Google COVID19 Mobility Report https://www.google.com/covid19/mobility/

Google Community Mobility Reports aim to provide insights into what has changed in response to policies aimed at combating COVID-19. The data is updated daily. It contains id information (including state, city, county) with change in mobility in percent with respect to a reference time interval, 3 Jan - 6 Feb 2020. The change is divided into six categories: 
<ul>
<li>'retail_and_recreation_percent_change_from_baseline',</li>
<li>'grocery_and_pharmacy_percent_change_from_baseline',</li>
<li>'parks_percent_change_from_baseline',</li>
<li>'transit_stations_percent_change_from_baseline',</li>
<li>'workplaces_percent_change_from_baseline',</li>
<li>'residential_percent_change_from_baseline'</li>
</ul>

Beginning with reading the raw data:

In [43]:
df = pd.read_csv("../Traffic Studies/Global_Mobility_Report.csv")

/var/folders/k5/yg__kmls5zndgtyl75h907040000gn/T/ipykernel_1152/4183005954.py:1: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../Traffic Studies/Global_Mobility_Report.csv")


<br>

Let's have a first look.

In [44]:
df.head()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0


As seen from the above table, there are detailed location information along with mobility change in percent in six categories.

We should check the data types.

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10430715 entries, 0 to 10430714
Data columns (total 15 columns):
 #   Column                                              Dtype  
---  ------                                              -----  
 0   country_region_code                                 object 
 1   country_region                                      object 
 2   sub_region_1                                        object 
 3   sub_region_2                                        object 
 4   metro_area                                          object 
 5   iso_3166_2_code                                     object 
 6   census_fips_code                                    float64
 7   place_id                                            object 
 8   date                                                object 
 9   retail_and_recreation_percent_change_from_baseline  float64
 10  grocery_and_pharmacy_percent_change_from_baseline   float64
 11  parks_percent_change_from_baseline 

Everything except date looks in the correct format. The next step is to fix that issue.

In [46]:
df["date"] = pd.DatetimeIndex(df["date"])

Now, we should not have a problem with the format. Let's check analytics of this dataframe.

In [47]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
census_fips_code,2221674.0,31007.348313,16043.096431,1001.0,18129.0,29161.0,46065.0,72153.0
retail_and_recreation_percent_change_from_baseline,6459698.0,-9.552462,30.139092,-100.0,-25.0,-8.0,7.0,741.0
grocery_and_pharmacy_percent_change_from_baseline,6161319.0,11.788329,33.319320,-100.0,-5.0,7.0,25.0,1064.0
parks_percent_change_from_baseline,4918818.0,5.636343,61.011146,-100.0,-31.0,-5.0,28.0,1327.0
transit_stations_percent_change_from_baseline,5172388.0,-14.804352,37.065404,-100.0,-39.0,-18.0,4.0,794.0
workplaces_percent_change_from_baseline,10059680.0,-11.023142,25.413069,-100.0,-25.0,-13.0,1.0,450.0
residential_percent_change_from_baseline,6400246.0,6.816356,7.342154,-77.0,2.0,6.0,10.0,75.0


Cencus Fips Code is a geography related indicator that we do not require. The other variables are mobility change in percent. They look in the correct range. Looking at the count, we can say that there are a lot of missing data. Let's check that.

In [48]:
df.isnull().sum()

country_region_code                                       6500
country_region                                               0
sub_region_1                                            172445
sub_region_2                                           1695920
metro_area                                            10374686
iso_3166_2_code                                        8611394
census_fips_code                                       8209041
place_id                                                 19206
date                                                         0
retail_and_recreation_percent_change_from_baseline     3971017
grocery_and_pharmacy_percent_change_from_baseline      4269396
parks_percent_change_from_baseline                     5511897
transit_stations_percent_change_from_baseline          5258327
workplaces_percent_change_from_baseline                 371035
residential_percent_change_from_baseline               4030469
dtype: int64

The missing values in the geographical values may be due to unability to locate the users. It is better not to drop due to these variables. Because it may result loosing too many valuable information. For example, dropping a value duw to unidentified county (sub_region_2) would be an unnecessary sacrifice, because this value can be used in country and city (sub_region_1) analysis. Simalarly, missing values of the mobility changes can be ignored, because if one of the six values is missing, it would be lose of other five variables. 

These missing values should be deleted or imputed when we apply machine learning algorithms, which we will not cover in this particular work.

Before we finish, we can add a new column for the mean of the mobility changes.

In [50]:
df["mean_percent_change"]=df.iloc[:,9:15].mean(axis=1)

<br><br><br>

#### 3.2. GDP per capita

<b>Source: </b>World Bank

We start by reading and looking at the data.

In [51]:
df_gdp = pd.read_csv("GDP_world_bank.csv", sep='"",""')

/var/folders/k5/yg__kmls5zndgtyl75h907040000gn/T/ipykernel_1152/182171546.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_gdp = pd.read_csv("GDP_world_bank.csv", sep='"",""')


In [52]:
df_gdp.head()

,"Country Name,""Country Code",Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,"2021"""","""
0,"""Aruba,""""ABW",GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,24712.493263,26441.619936,26893.011506,28396.908423,28452.170615,29350.805019,30253.279358,NaN,NaN,""""","""
1,"""Africa Eastern and Southern,""""AFE",GDP per capita (current US$),NY.GDP.PCAP.CD,153.494439,152.873313,162.413188,189.506170,168.816995,187.396306,198.450501,...,1736.166560,1713.899299,1703.596298,1549.037940,1431.778723,1573.063386,1574.978648,1530.059177,1359.618224,""""","""
2,"""Afghanistan,""""AFG",GDP per capita (current US$),NY.GDP.PCAP.CD,59.773234,59.860900,58.458009,78.706429,82.095307,101.108325,137.594298,...,638.845852,624.315455,614.223342,556.007221,512.012778,516.679862,485.668419,494.179350,516.747871,""""","""
3,"""Africa Western and Central,""""AFW",GDP per capita (current US$),NY.GDP.PCAP.CD,107.932233,113.081647,118.831107,123.442888,131.854402,138.526332,144.326212,...,1965.118485,2157.481149,2212.853135,1894.310195,1673.835527,1613.473553,1704.139603,1777.918672,1710.073363,""""","""
4,"""Angola,""""AGO",GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5100.097027,5254.881126,5408.411700,4166.979833,3506.073128,4095.810057,3289.643995,2809.626088,1776.166868,""""","""


In [53]:
df_gdp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 65 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Country Name,"Country Code  266 non-null    object 
 1   Indicator Name              266 non-null    object 
 2   Indicator Code              266 non-null    object 
 3   1960                        132 non-null    float64
 4   1961                        134 non-null    float64
 5   1962                        137 non-null    float64
 6   1963                        137 non-null    float64
 7   1964                        137 non-null    float64
 8   1965                        148 non-null    float64
 9   1966                        151 non-null    float64
 10  1967                        154 non-null    float64
 11  1968                        159 non-null    float64
 12  1969                        159 non-null    float64
 13  1970                        168 non

This time we are so lucky about cleanness of the data. Also the 2021 data seems to be missing. So we should first fix the country names for merging with our dataframe. Then we can only take GDP data of 2020, because our dataframe contains the time interval from 2020 to 2022. But first, better start with cleaning column names.

In [54]:
df_gdp.columns

Index(['Country Name,"Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021"","'],
      dtype='object')

We copy the column names and fix them, at least for country names. This is optional.

In [55]:
df_gdp.columns = ['Country Name', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

Now we clean the country names.

In [56]:
df_gdp["Country Name"] = df_gdp["Country Name"].apply(lambda x: x.split(",")[0].strip('"'))

In [57]:
df_gdp.head()

,Country Name,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,24712.493263,26441.619936,26893.011506,28396.908423,28452.170615,29350.805019,30253.279358,NaN,NaN,""""","""
1,Africa Eastern and Southern,GDP per capita (current US$),NY.GDP.PCAP.CD,153.494439,152.873313,162.413188,189.506170,168.816995,187.396306,198.450501,...,1736.166560,1713.899299,1703.596298,1549.037940,1431.778723,1573.063386,1574.978648,1530.059177,1359.618224,""""","""
2,Afghanistan,GDP per capita (current US$),NY.GDP.PCAP.CD,59.773234,59.860900,58.458009,78.706429,82.095307,101.108325,137.594298,...,638.845852,624.315455,614.223342,556.007221,512.012778,516.679862,485.668419,494.179350,516.747871,""""","""
3,Africa Western and Central,GDP per capita (current US$),NY.GDP.PCAP.CD,107.932233,113.081647,118.831107,123.442888,131.854402,138.526332,144.326212,...,1965.118485,2157.481149,2212.853135,1894.310195,1673.835527,1613.473553,1704.139603,1777.918672,1710.073363,""""","""
4,Angola,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5100.097027,5254.881126,5408.411700,4166.979833,3506.073128,4095.810057,3289.643995,2809.626088,1776.166868,""""","""


That is better. Now we can merge 2020 GDP per capita values to our mobility dataframe.

In [58]:
df = df.merge(df_gdp[["Country Name", "2020"]], how="left", left_on="country_region", right_on="Country Name")

In [59]:
df.head()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,mean_percent_change,Country Name,2020
0,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0,2.000000,United Arab Emirates,36284.555243
1,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0,2.166667,United Arab Emirates,36284.555243
2,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0,1.500000,United Arab Emirates,36284.555243
3,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0,1.166667,United Arab Emirates,36284.555243
4,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0,0.666667,United Arab Emirates,36284.555243


Our first merging dataframes mission seems to be successful. Now let's change the column name and drop the extra country name.

In [60]:
df.columns

Index(['country_region_code', 'country_region', 'sub_region_1', 'sub_region_2',
       'metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id', 'date',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'mean_percent_change',
       'Country Name', '2020'],
      dtype='object')

In [62]:
df.columns = ['country_region_code', 'country_region', 'sub_region_1', 'sub_region_2',
       'metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id', 'date',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline','mean_percent_change', 'Country Name', 'gdp_per_cap_2020']

In [63]:
df.drop("Country Name",axis=1, inplace=True)

<br><br><br>

#### 3.3. COVID19 Case Reports

<b>Source:</b> The data was collected from https://ourworldindata.org/coronavirus-data

It is time to look at COVID19 incidents.

In [64]:
df_cases = pd.read_csv("../Traffic Studies/owid-covid-data.csv")

In [65]:
df_cases.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [66]:
df_cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198617 entries, 0 to 198616
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    198617 non-null  object 
 1   continent                                   187101 non-null  object 
 2   location                                    198617 non-null  object 
 3   date                                        198617 non-null  object 
 4   total_cases                                 190685 non-null  float64
 5   new_cases                                   190367 non-null  float64
 6   new_cases_smoothed                          189190 non-null  float64
 7   total_deaths                                172058 non-null  float64
 8   new_deaths                                  172012 non-null  float64
 9   new_deaths_smoothed                         170854 non-null  float64
 

<br>

The dataframe seems clean, yet, the date should be formatted. There are also too many columns. So we can pick only three to add our dataframe. Also, there are some missing values. This is mostly because either the number is too small, e.g. x per millions values, or the missing values are in fact zero. Therefore, we can fill them with zero for the number of incidents.

In [67]:
df_cases["date"] = pd.DatetimeIndex(df_cases["date"])

In [68]:
df_cases["total_cases_per_million"] = df_cases["total_cases_per_million"].fillna(0)
df_cases["new_deaths_per_million"] = df_cases["new_deaths_per_million"].fillna(0)
df_cases["hosp_patients_per_million"] = df_cases["hosp_patients_per_million"].fillna(0)

In [69]:
df = df.merge(df_cases[["continent","location","date",
                        "new_cases_per_million","total_cases_per_million","new_deaths_per_million",
                        "hosp_patients_per_million"]], 
              how="left", left_on=["country_region","date"], right_on=["location","date"])

In [70]:
df.drop("location", axis=1, inplace=True)

<br><br><br>

#### 3.4. Weather (Precipitation)

<b>Source: </b>The Royal Netherlands Meteorological Institute (KNMI)

Precipitation values for a Turkish City, Bursa, is collected. The collected data was in txt format. We simply copy and paste it into excel and load the data.

In [71]:
bursa_yagis = pd.read_excel("../Traffic Studies/Benzin_fiyatlari_opet.xlsx", sheet_name="bursa_yagis")

In [72]:
bursa_yagis.head()

,year,month,day,precipitation [mm/day]
0,1973,1,6,0.0
1,1973,1,7,0.0
2,1973,1,8,0.0
3,1973,1,9,0.0
4,1973,1,10,0.0


In [73]:
bursa_yagis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13782 entries, 0 to 13781
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    13782 non-null  int64  
 1   month                   13782 non-null  int64  
 2   day                     13782 non-null  int64  
 3   precipitation [mm/day]  13782 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 430.8 KB


In [79]:
bursa_yagis.isnull().sum()

precipitation [mm/day]    0
date                      0
dtype: int64

<br>

The data is clean, however, to match the dates with our dataframe a new date column is required. 

In [74]:
bursa_yagis["date"]=pd.to_datetime(bursa_yagis[["year","month","day"]])

In [75]:
bursa_yagis.drop(["year", "month", "day"], axis=1,inplace=True)

In [76]:
bursa_yagis.head()

,precipitation [mm/day],date
0,0.0,1973-01-06
1,0.0,1973-01-07
2,0.0,1973-01-08
3,0.0,1973-01-09
4,0.0,1973-01-10


Let's add another column that is boolean to show if there is any rain on that particular day. This may be useful, since people may classifies the weather simply rainy or not and decide on mobility related choices.

In [80]:
bursa_yagis["rain"] = bursa_yagis["precipitation [mm/day]"].apply(lambda x: "Rainy" if x>0 else "No rain")

In [81]:
bursa_yagis

,precipitation [mm/day],date,rain
0,0.0,1973-01-06,No rain
1,0.0,1973-01-07,No rain
2,0.0,1973-01-08,No rain
3,0.0,1973-01-09,No rain
4,0.0,1973-01-10,No rain
...,...,...,...
13777,0.0,2021-09-21,No rain
13778,0.0,2021-09-22,No rain
13779,3.0,2021-09-23,Rainy
13780,2.5,2021-09-24,Rainy


Lastly, we should add another column to match the city with our original dataframe.

In [83]:
bursa_yagis["sub_region_1"] = "Bursa"

Okay, now we can merge the dataframes.

In [84]:
df = df.merge(bursa_yagis, how="left", on = ["date", "sub_region_1"])

<br><br><br>

#### 3.5. Traffic Congestion

<b>Source: </b> İBB Açık Veri

Since there is no dataframe covering all traffic congestion info for all world, we selected Istanbul as our sample. Istanbul Metropolitan Municipality (IBB) publishes traffic congestion index, so we begin with scraping this from its website. The data is held in a datastore service and served in json format. We can find the corresponding link from the manual on that website.

In [87]:
url = "https://data.ibb.gov.tr/en/datastore/odata3.0/ba47eacb-a4e1-441c-ae51-0e622d4a18e2?$top=910&$format=json"
fileobj = urllib.urlopen(url)
data= json.loads(fileobj.read())
df_traffic_ist = pd.DataFrame(data["value"])
df_traffic_ist.tail()

,_id,TRAFFIC_INDEX_DATE,MINIMUM_TRAFFIC_INDEX,MAXIMUM_TRAFFIC_INDEX,AVERAGE_TRAFFIC_INDEX
901,902,2021-06-25 00:00:00 +0000 UTC,1,63,33
902,903,2021-06-26 00:00:00 +0000 UTC,1,255,35
903,904,2021-06-27 00:00:00 +0000 UTC,1,15,4
904,905,2021-06-28 00:00:00 +0000 UTC,1,255,58
905,906,2021-06-29 00:00:00 +0000 UTC,1,255,70


In [88]:
df_traffic_ist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 906 entries, 0 to 905
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   _id                    906 non-null    int64 
 1   TRAFFIC_INDEX_DATE     906 non-null    object
 2   MINIMUM_TRAFFIC_INDEX  906 non-null    int64 
 3   MAXIMUM_TRAFFIC_INDEX  906 non-null    int64 
 4   AVERAGE_TRAFFIC_INDEX  906 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 35.5+ KB


<br>

The dataframe contains only four columns, howeveri the date looks quite bad. First, we need to clean '+0000 UTC' parts, then reformat it as date.

In [90]:
strs = df_traffic_ist['TRAFFIC_INDEX_DATE'].apply(lambda x: re.sub(' \w+$','',x))
df_traffic_ist['TRAFFIC_INDEX_DATE']=pd.to_datetime( strs ).dt.strftime('%Y-%m-%d')
df_traffic_ist['TRAFFIC_INDEX_DATE']=pd.to_datetime(df_traffic_ist['TRAFFIC_INDEX_DATE'])

In [91]:
df_traffic_ist.drop("_id", axis=1,inplace=True)

In [92]:
df_traffic_ist.head()

,TRAFFIC_INDEX_DATE,MINIMUM_TRAFFIC_INDEX,MAXIMUM_TRAFFIC_INDEX,AVERAGE_TRAFFIC_INDEX
0,2019-01-01,1,31,11
1,2019-01-02,1,58,24
2,2019-01-03,1,255,31
3,2019-01-04,8,60,28
4,2019-01-05,2,255,26


<br>

Now it looks good. To merge with our main dataframe, we can make some minor changes and add a sub_region column.

In [93]:
df_traffic_ist.columns = ['date', 'MINIMUM_TRAFFIC_INDEX', 'MAXIMUM_TRAFFIC_INDEX',
       'AVERAGE_TRAFFIC_INDEX']

In [94]:
df_traffic_ist["sub_region_1"] = "İstanbul"

In [95]:
df = df.merge(df_traffic_ist, how="left", on=["date", "sub_region_1"])

<br><br><br>

#### 3.6. City/County Development Index

<b>Source:</b> https://www.sanayi.gov.tr/merkez-birimi/b94224510b7b/sege

Development index is an indicator published by governmental institutions for the cities and counties as well. There is a serious challenge to get this data. The data is publicly available but in PDF format that does not only contain tables. So we can follow this steps:

<ul>
<li>First, convert PDF to Word File. Online tools like smallpdf.com may handle this.</li>
<li>Copy the tables one by one and paste them in an excel sheet.</li>
<li>Clean the excel sheet and read the final sheet.</li>
</ul>

After these steps, we can merge the city index and county index in one table and merge them with our main dataframe.

In [100]:
temp = pd.read_excel("tr_gelismislik.xlsx", sheet_name="2017_il_sege")

In [101]:
temp.head()

,Sıra,İl Adı,Skor,Kademe
0,1,İstanbul,4.051,1
1,2,Ankara,2.718,1
2,3,İzmir,1.926,1
3,4,Kocaeli,1.787,1
4,5,Antalya,1.642,1


In [102]:
temp2 = pd.read_excel("tr_gelismislik.xlsx", sheet_name="2022_ilce_sege")

In [103]:
temp2.head()

,Sıra,İl Adı,İlçe Adı,Skor,Kademe
0,1,İstanbul,Şişli,6.959,1
1,2,Ankara,Çankaya,6.901,1
2,3,İstanbul,Beşiktaş,5.940,1
3,4,İstanbul,Kadıköy,4.910,1
4,5,Ankara,Yenimahalle,4.481,1


In [104]:
tr_gelismislik = temp2.merge(temp,how="left", on="İl Adı")

In [105]:
tr_gelismislik.columns = ['Sıra_x', 'il', 'ilce', 'ilce_skor', 'ilce_kademe', 'Sıra_y',
       'il_skor', 'il_kademe']

In [106]:
tr_gelismislik.drop(["Sıra_x", "Sıra_y"], axis=1, inplace=True)

In [107]:
tr_gelismislik.dropna(axis=0, inplace=True)

In [108]:
tr_gelismislik["il_kademe"] = tr_gelismislik["il_kademe"].astype("category")
tr_gelismislik["ilce_kademe"] = tr_gelismislik["ilce_kademe"].astype("category")

In [109]:
tr_gelismislik.head()

,il,ilce,ilce_skor,ilce_kademe,il_skor,il_kademe
0,İstanbul,Şişli,6.959,1,4.051,1.0
1,Ankara,Çankaya,6.901,1,2.718,1.0
2,İstanbul,Beşiktaş,5.940,1,4.051,1.0
3,İstanbul,Kadıköy,4.910,1,4.051,1.0
4,Ankara,Yenimahalle,4.481,1,2.718,1.0


<br>

<b>Important Note:</b> These kind of data, especially local data, should be carefully checked, because there maybe unexpected errors. This is why we still need human-intelligence before artificial-intelligence.

<br>Still we need some minor changes with the names. These minor changes are very important. For example, "Merkez" is not an actual county name, it means "center" in English. However, in our main dataframe it is joined with the city names, so instead of "Merkez", it says "Bursa Merkez". So this would be a problem while merging. We should fix that.

In [111]:
tr_gelismislik[tr_gelismislik["ilce"] == "Merkez"]

,il,ilce,ilce_skor,ilce_kademe,il_skor,il_kademe
39,Edirne,Merkez,2.113,1,0.534,2.0
52,Çanakkale,Merkez,1.846,1,0.548,2.0
72,Isparta,Merkez,1.551,2,0.564,2.0
82,Bolu,Merkez,1.480,2,0.760,2.0
96,Kütahya,Merkez,1.230,2,0.170,3.0
100,Uşak,Merkez,1.204,2,0.278,3.0
101,Zonguldak,Merkez,1.189,2,0.333,3.0
103,Afyonkarahisar,Merkez,1.178,2,-0.023,4.0
108,Giresun,Merkez,1.145,2,-0.323,5.0
109,Sivas,Merkez,1.138,2,-0.137,4.0


In [112]:
df["sub_region_2"] = df["sub_region_2"].apply(lambda x: "Merkez" if "Merkez" in str(x) else x)

There is also special characters problems.

In [113]:
df["sub_region_2"] = df["sub_region_2"].apply(lambda x: "Kahta" if "Kâhta" in str(x) else x)

In [114]:
df = df.merge(tr_gelismislik, how="left", left_on=["sub_region_1", "sub_region_2"], right_on=["il", "ilce"])

In [115]:
df[df["sub_region_1"] == "İstanbul"].tail()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,...,rain,MINIMUM_TRAFFIC_INDEX,MAXIMUM_TRAFFIC_INDEX,AVERAGE_TRAFFIC_INDEX,il,ilce,ilce_skor,ilce_kademe,il_skor,il_kademe
7793692,TR,Turkey,İstanbul,Zeytinburnu,NaN,NaN,NaN,ChIJe2qviqi7yhQRkPxCar2Jtr4,2022-06-25,-10.0,...,NaN,NaN,NaN,NaN,İstanbul,Zeytinburnu,2.382,1,4.051,1.0
7793693,TR,Turkey,İstanbul,Zeytinburnu,NaN,NaN,NaN,ChIJe2qviqi7yhQRkPxCar2Jtr4,2022-06-26,-11.0,...,NaN,NaN,NaN,NaN,İstanbul,Zeytinburnu,2.382,1,4.051,1.0
7793694,TR,Turkey,İstanbul,Zeytinburnu,NaN,NaN,NaN,ChIJe2qviqi7yhQRkPxCar2Jtr4,2022-06-27,-1.0,...,NaN,NaN,NaN,NaN,İstanbul,Zeytinburnu,2.382,1,4.051,1.0
7793695,TR,Turkey,İstanbul,Zeytinburnu,NaN,NaN,NaN,ChIJe2qviqi7yhQRkPxCar2Jtr4,2022-06-28,-7.0,...,NaN,NaN,NaN,NaN,İstanbul,Zeytinburnu,2.382,1,4.051,1.0
7793696,TR,Turkey,İstanbul,Zeytinburnu,NaN,NaN,NaN,ChIJe2qviqi7yhQRkPxCar2Jtr4,2022-06-29,0.0,...,NaN,NaN,NaN,NaN,İstanbul,Zeytinburnu,2.382,1,4.051,1.0


<br><br><br>

#### 3.7. Fuel Prices

<b>Source: </b> Opet, https://www.opet.com.tr/akaryakit-fiyatlari-arsivi

In the recent years the fuel prices increased dramatically. Especially, in Turkey, where the local currency lost a significant value against dollar, pushed the prices even higher. Therefore, fuel prices in Turkey is added to the dataframe to investigate if the increasing fuel prices has an effect on the mobility behaviour. This data is not given as csv or excel. Rather, we should scrap it from the web site by just copying and pasting in an excel sheet.

In [116]:
benzin_fiyatlari = pd.read_excel("Benzin_fiyatlari_opet.xlsx",sheet_name = "benzin_fiyatlari")

In [118]:
benzin_fiyatlari.head()

,Tarih,KDV,Birim,Kurşunsuz Benzin,Motorin
0,2020-02-18,(KDV'li),TL/LT,6.66,6.37
1,2020-02-25,(KDV'li),TL/LT,6.83,6.37
2,2020-02-26,(KDV'li),TL/LT,6.73,6.37
3,2020-02-27,(KDV'li),TL/LT,6.73,6.37
4,2020-02-29,(KDV'li),TL/LT,6.57,6.24


In [117]:
benzin_fiyatlari.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Tarih             314 non-null    datetime64[ns]
 1   KDV               314 non-null    object        
 2   Birim             314 non-null    object        
 3   Kurşunsuz Benzin  314 non-null    float64       
 4   Motorin           314 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 12.4+ KB


<br><br>

The dataframe seems clean. However, there is another issue that we should handle. This dataframe includes the dates only when the price changes. When merging with the main dataframe, the other days will be missing. To handle that, first, we need to have a proper date column. This can be acquired from the main dataframe. Then we can fill the missing values with the most recent filled values.

In [119]:
date_df = pd.DataFrame(df["date"].unique(), columns=["date"]).sort_values("date")

In [120]:
df_benzin_fiyatlari = date_df.merge(benzin_fiyatlari,how="left", left_on="date", right_on="Tarih")

In [121]:
df_benzin_fiyatlari = df_benzin_fiyatlari.fillna(method="ffill")

In [123]:
df_benzin_fiyatlari.head()

,date,Tarih,KDV,Birim,Kurşunsuz Benzin,Motorin
0,2020-02-15,2020-02-18,(KDV'li),TL/LT,6.66,6.37
1,2020-02-16,2020-02-18,(KDV'li),TL/LT,6.66,6.37
2,2020-02-17,2020-02-18,(KDV'li),TL/LT,6.66,6.37
3,2020-02-18,2020-02-18,(KDV'li),TL/LT,6.66,6.37
4,2020-02-19,2020-02-18,(KDV'li),TL/LT,6.66,6.37


Now we add country value to match with the main dataframe, then we are good to go.

In [124]:
df_benzin_fiyatlari["country_region"] = "Turkey"

In [125]:
df_benzin_fiyatlari.drop(["KDV","Birim"], axis=1, inplace=True)

In [126]:
df_benzin_fiyatlari.drop("Tarih", axis=1, inplace=True)

In [127]:
df = df.merge(df_benzin_fiyatlari, how="left", on=["date","country_region"])

In [128]:
df.head()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,...,MAXIMUM_TRAFFIC_INDEX,AVERAGE_TRAFFIC_INDEX,il,ilce,ilce_skor,ilce_kademe,il_skor,il_kademe,Kurşunsuz Benzin,Motorin
0,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-15,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-16,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-17,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-18,-2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-19,-2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
df[df["country_region"] == "Turkey"].tail()

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,...,MAXIMUM_TRAFFIC_INDEX,AVERAGE_TRAFFIC_INDEX,il,ilce,ilce_skor,ilce_kademe,il_skor,il_kademe,Kurşunsuz Benzin,Motorin
8031771,TR,Turkey,Zonguldak,Merkez,NaN,NaN,NaN,ChIJ5RJc34yHnEAR6s60uOmblJc,2022-06-25,-24.0,...,NaN,NaN,Zonguldak,Merkez,1.189,2,0.333,3.0,27.26,29.99
8031772,TR,Turkey,Zonguldak,Merkez,NaN,NaN,NaN,ChIJ5RJc34yHnEAR6s60uOmblJc,2022-06-26,-37.0,...,NaN,NaN,Zonguldak,Merkez,1.189,2,0.333,3.0,27.26,29.99
8031773,TR,Turkey,Zonguldak,Merkez,NaN,NaN,NaN,ChIJ5RJc34yHnEAR6s60uOmblJc,2022-06-27,-44.0,...,NaN,NaN,Zonguldak,Merkez,1.189,2,0.333,3.0,27.26,29.99
8031774,TR,Turkey,Zonguldak,Merkez,NaN,NaN,NaN,ChIJ5RJc34yHnEAR6s60uOmblJc,2022-06-28,-25.0,...,NaN,NaN,Zonguldak,Merkez,1.189,2,0.333,3.0,27.26,29.99
8031775,TR,Turkey,Zonguldak,Merkez,NaN,NaN,NaN,ChIJ5RJc34yHnEAR6s60uOmblJc,2022-06-29,-22.0,...,NaN,NaN,Zonguldak,Merkez,1.189,2,0.333,3.0,27.30,27.51


<br><br><br>

### 4. Conclusion

In this work, various data from multiple sources are acquired and merged together. Some data are downloaded as csv files and read easily, whereas, some data are requested in json from datastores, some are acquired using web scraping and some can only acquired by PDF file with primitive manipulation methods. As seen in this work, not only collecting data but cleaning is very important. Especially, dealing with local names and special characters require human attention to be spotted. 

<li>This dataframe is used in machine learning algorithm studies like regression and clustering. The link will be provided as soon as the files are uploaded in this repository.</li>

<li>The dataframe is also used in data visualization projects. 

Tableau Data Visualization: https://github.com/DrFarukAydin/data-science-portfolio/blob/main/data-visualization-tableau/data-visualization-tableau.ipynb 

Python Data Visualization: The link will be provided as soon as the files are uploaded in this repository.

<br><br><br>

<br><br><br>